# A Script for Loading and Processing .csv Data

## by Tan Tran

Tutorial for sktime interview

## Example scenario: 
* Given time series Case A.csv...CaseH.csv
* For features ending with :PMAX.HVAL, report the max values each feature.
* For features ending with _F_LBF, report the max and min values for each feature/case and case resulting in those value


* Problem: Using Excel for loading and processing .csv is not practical.
    1. Excel requires extreme labor
    2. `VBA`, even worse

* Goal:
    1. Perform load, extract and process
    2. Expected result: User-friendly and Excel-friendly DataFrame or equivalent
    3. Plots

* Solution: Use `pandas`

In [1]:
import warnings
warnings.filterwarnings('ignore')

import os
import sktime
import pandas as pd
import numpy as np


## Implementation

## Structure


1. Master Class: `prep_csv`
2. Public method: `prep_csv()` and `update()`
3. Internal methods: `_load_data()`, `_prep_max_min()`, `_add_bool_to_max_min()`,
`_prep_force()`, `_prep_mm()`

In [2]:
import os
import numpy as np
import pandas as pd
from tqdm.notebook import tqdm

import warnings
warnings.filterwarnings('ignore')

class prep_csv:
    def __init__(
        self,
        path=None,
        file_format=".csv",
        dtype=np.float32,
        decimal=2,
        force_ending="_F_LBF",
        custom_ending="MM_",
        custom_attribute=None
    ):
        self.path=path
        self.file_format=file_format
        self.dtype=dtype
        self.decimal=decimal
        self.force_ending=force_ending
        self.custom_ending=custom_ending
        self.custom_attribute=custom_attribute
        # Set precision display
        decimal_point='{:,.'+str(self.decimal)+'f}'
        pd.options.display.float_format = decimal_point.format
        super().__init__()

        # DataFrame-style report
        self.force_report=None
        self.mm_report=None
        # Internal variables
        self.segment=None
        self.all_data=None
        self.fmax=None
        self.fmin=None
        self.all_max=None
        self.all_min=None
        self.custom_attribute_report=None
        self.selected_case=None

    def prep_csv(self):
        self._load_data()
        self._prep_max_min()
        self._add_bool_to_max_min()
        self._prep_force()
        self._prep_mm()
        if isinstance(self.custom_attribute, type(None)) is not True:
            self._get_custom_attribute()
        return self

    def update(self):
        self._filter_case()
        self._prep_force()
        self._prep_mm()
        if isinstance(self.custom_attribute, type(None)) is not True:
            self._get_custom_attribute()
        print("Update finished!")

    def _load_data(self):
        """
        Load all data into one Super MultiIndex DataFrame
        return: A multiIndex DataFrame of all files 
        Index level 0: case name
        Index level 1: time series
        Return: A MultiIndex DataFrame containing all data

        To "extract" dataframe for a specific case, use the formula
        ResultMax.filter(items=['case name'], axis=0)
        
        Parameter:
        path: absolute or relative path to the folder where csv files are
        file_format: only suport csv at the moment
        dtype: numpy dtype, np.float16, np.float32 or np.float64
        decimal: number of decimal points to display
        force_ending: the suffix or prefix used to identify force
        custom_ending: the suffix or prefix used to identify custom feature (pressure)
        custom_attribute: the suffix or prefix used to identify other feature, without processing max-min

        How to use:
        1/ Instantiate the class with parameters
        2/ Call class.prep_csv()
        3/ If needed, remove some cases by assessing the INCLUDE boolean with class.selected_case.
        Set the non-needed case to be zero
        4/ Update the inner state with class.update() 

        Attribute:
        fmax, fmin: Maximum or minimum force in the segment. If minimum, the values will be absolute value.
        force_report: Maximum force and associated case.
        mm_report: Maximum pressure ans associated case.

        """
        # Get a list of files
        file_list = os.listdir(self.path)
        chosen_files = list(filter(lambda f: f.endswith(self.file_format), file_list))
        num_files = len(chosen_files)
        case = []
        time = []
        columns = []
        #content = []
        for i, file in enumerate(tqdm(chosen_files, bar_format='{desc}{percentage:3.0f}%{r_bar}')):
            case_name = file.replace(self.file_format,"")
            # Read each file, first loop is different because we do not know column labels yet.
            current_file = pd.read_csv(path + file)
            if i==0:
                print(f"Now working on {case_name}...")
                columns = np.array(current_file.columns[1:], dtype="U200")
                columns = np.char.strip(columns)
                content = np.empty([1,len(current_file.columns)-1])
                current_file = current_file.drop([0])
                new_index = np.arange(len(current_file))  # Reset index to 0
                current_file = current_file.reindex(new_index, method="backfill")
                # For each file, we get, case name, time array, content array, column labels
                current_time = current_file.iloc[:,0].astype(self.dtype).to_numpy()  # Get time array
                current_content = current_file.iloc[:,1:].astype(self.dtype).to_numpy()  # Get content array
                # Get case name array to have the same length as time series in order to create MultiIndex
                current_case = np.empty(len(current_time), dtype='U200')
                current_case.fill(case_name)
                # Append all arays
                case = np.append(case, current_case)
                time = np.append(time, current_time)
                content = np.concatenate((content, current_content), axis=0)
                content = np.delete(content, 0, 0)  # Delete first row because it's an empty array
                print(f"Finished {case_name}")
            else:
                case_name = file.replace(self.file_format,"")
                print(f"Now working on {case_name}...")
                current_file = pd.read_csv(self.path + file)
                current_file = current_file.drop([0])
                new_index = np.arange(len(current_file))  # Reset index to 0
                current_file = current_file.reindex(new_index, method="backfill")
                current_time = current_file.iloc[:,0].astype(self.dtype).to_numpy()  # Get time array
                current_content = current_file.iloc[:,1:].astype(self.dtype).to_numpy()  # Get content array
                # Get case array to have the same length as time series in order to create MultiIndex
                current_case = np.empty(len(current_time), dtype="U200")
                current_case.fill(case_name)
                # Append all arays
                case = np.append(case, current_case)
                time = np.append(time, current_time)
                content = np.concatenate((content, current_content), axis=0)
                print(f"Finished {case_name}")

        # Create MultiIndex array    
        index_array = [
            np.array(case),  
            np.array(time),  
        ]
        tuples = list(zip(*index_array))
        index = pd.MultiIndex.from_tuples(tuples, names=["CASE", "TIME"])
        # Create final DataFrame
        all_cases = pd.DataFrame(
        content, 
        index=index, 
        columns=columns)        
        self.all_data=all_cases.astype(self.dtype) 
        return      
    
    def _prep_max_min(self):
        # Create 2 DataFrame: Max and Min 
        self.all_max=self.all_data.groupby(level=0).agg(["max"]).droplevel(1, axis=1)  # Max values of each case (level 0)
        self.all_min=self.all_data.groupby(level=0).agg(["min"]).droplevel(1, axis=1)  # Min values of each case (level 0)
        return self

    def _prep_force(self):
        # Find maximum/minimum of force/feature out of all cases and the case associated with it  
        all_data_max=self.all_max
        all_data_max=all_data_max[all_data_max["INCLUDE"]>0]  # Only keep cases whose "INCLUDE" value is 1
        all_data_min=self.all_min
        all_data_min=all_data_min[all_data_min["INCLUDE"]>0]
        # Apply masking to extract force
        selected_columns = list(filter(lambda f: f.endswith(self.force_ending) or f.startswith(self.force_ending), all_data_max.columns))
        clean_columns = [col.replace(self.force_ending, "") for col in selected_columns]
        # Process max
        selected_force_max = all_data_max[selected_columns].apply(lambda x: x/1000)
        selected_force_max.columns = clean_columns
        self.segment = clean_columns
        df_force_max = selected_force_max.max().apply(lambda x: 0 if x<0 else x)
        # Apply mask to set negative value to 0
        x = selected_force_max > 0
        self.fmax = selected_force_max.where(x,0)
        df_force_max=df_force_max.to_dict()
        idx_force_max = selected_force_max.idxmax().to_dict()
        # Process min
        selected_force_min = all_data_min[selected_columns].apply(lambda x: x/(-1000))
        selected_force_min.columns = clean_columns
        df_force_min = selected_force_min.min().apply(lambda x: 0 if x<0 else x)
        # Apply mask to set negative value to 0
        x = selected_force_min > 0
        self.fmin = selected_force_min.where(x,0)
        df_force_min=df_force_min.to_dict()
        idx_force_min = selected_force_min.idxmin().to_dict()
        # Save result into a MultiIndex DataFrame
        header=[np.array(["With Flow", "Against Flow", "With Flow", "Against Flow"]), np.array(["kips", "kips", "Controlling Case", "Controlling Case"])]
        self.force_report=pd.DataFrame([df_force_max, df_force_min, idx_force_max, idx_force_min], index=header).T
        # Get worst force column and move it next to 'With Flow' column
        self.force_report[('Worst Force', 'kips')]=self.force_report[[('With Flow', 'kips'), ('Against Flow', 'kips')]].max(axis=1)
        move_col = self.force_report.pop(('Worst Force', 'kips'))
        self.force_report.insert(2, ('Worst Force', 'kips'), move_col)
        print("Processing Finished!")
        print("Against Flow is reported as absolute value. Actual Against Flow values are negative. If positive, it is set to be 0.")
        print("Likewise, if With Flow is negative, it is set to be 0.")
        return self
    
    def _prep_mm(self):
        "Process maximum pressure"
        all_data_max=self.all_max
        all_data_max=all_data_max[all_data_max["INCLUDE"]>0]  # Only keep cases whose "INCLUDE" value is 1
        # Apply masking to extract attribute
        selected_columns = list(filter(lambda f: f.endswith(self.custom_ending) or f.startswith(self.custom_ending), all_data_max.columns))
        clean_columns = [col.replace(self.custom_ending, "") for col in selected_columns]
        # Process max
        selected_max = all_data_max[selected_columns]
        selected_max.columns = clean_columns
        self.mm_report = selected_max
        return self

    def _get_custom_attribute(self):
        all_data=self.all_data
        # Apply masking to extract attribute
        selected_columns = list(filter(lambda f: f.endswith(self.custom_attribute) or f.startswith(self.custom_attribute), all_data.columns))
        clean_columns = [col.replace(self.custom_ending, "") for col in selected_columns]
        # Process max
        selected_data = all_data[selected_columns]
        selected_data.columns = clean_columns
        self.custom_attribute_report = selected_data
        return self

    # Method to be called by self.update(), allowing case selection
    def _add_bool_to_max_min(self):
        # Only call this once after load_data to add an "Include" column to all_max and all_min
        include_bool=np.empty(len(self.all_max), dtype=np.int32)
        include_bool.fill(1)
        self.all_max["INCLUDE"] = include_bool
        self.all_min["INCLUDE"] = include_bool
        self.selected_case = self.all_max[["INCLUDE"]]
        self.selected_case=self.selected_case.reset_index()
        return self
    
    def _filter_case(self):
        # called by update() when updating which case to include in the analysis
        self.all_max["INCLUDE"]=self.selected_case["INCLUDE"].values
        self.all_min["INCLUDE"]=self.selected_case["INCLUDE"].values
        return self

Instantiate the class

In [3]:
path = 'H:/Tensorflow Dataset/Example_CCLNG_ESD1/not used/'
PrepMyCSV = prep_csv(path=path, dtype=np.float32, decimal=2, force_ending="_F_LBF", custom_ending=":PMAX.HVAL")


## Call `prep_csv()` - Same as `fit()`

In [4]:
PrepMyCSV.prep_csv()

  0%| 0/52 [00:00<?, ?it/s]

Now working on dual_20s_Everywhere_14s_Jetty_10k...
Finished dual_20s_Everywhere_14s_Jetty_10k
Now working on dual_20s_Everywhere_14s_Jetty_12k...
Finished dual_20s_Everywhere_14s_Jetty_12k
Now working on dual_20s_Everywhere_14s_Tank_10k...
Finished dual_20s_Everywhere_14s_Tank_10k
Now working on dual_20s_Everywhere_14s_Tank_12k...
Finished dual_20s_Everywhere_14s_Tank_12k
Now working on dual_20s_Everywhere_16s_Jetty_10k...
Finished dual_20s_Everywhere_16s_Jetty_10k
Now working on dual_20s_Everywhere_16s_Jetty_12k...
Finished dual_20s_Everywhere_16s_Jetty_12k
Now working on dual_20s_Everywhere_16s_Tank_10k...
Finished dual_20s_Everywhere_16s_Tank_10k
Now working on dual_20s_Everywhere_16s_Tank_12k...
Finished dual_20s_Everywhere_16s_Tank_12k
Now working on dual_20s_Everywhere_18s_Jetty_10k...
Finished dual_20s_Everywhere_18s_Jetty_10k
Now working on dual_20s_Everywhere_18s_Jetty_12k...
Finished dual_20s_Everywhere_18s_Jetty_12k
Now working on dual_20s_Everywhere_18s_Tank_10k...
Finishe

All files are now in panel-type data

In [5]:
PrepMyCSV.all_data

XV24541:P-  XV24521:P-  XV24501:P-  \
CASE                              TIME                                       
dual_20s_Everywhere_14s_Jetty_10k 0.00       49.98       49.92       49.87   
                                  0.00       49.98       49.92       49.87   
                                  0.00       49.98       49.92       49.87   
                                  0.00       49.98       49.92       49.87   
                                  0.00       49.98       49.92       49.87   
...                                            ...         ...         ...   
single_22s_all_XV_12k             7.17      127.25      127.25      127.25   
                                  7.25      127.25      127.25      127.25   
                                  7.33      127.25      127.25      127.25   
                                  7.43      127.25      127.25      127.25   
                                  7.57      127.25      127.25      127.25   

                                        XV24641:P-  XV24621:P-  XV24601:P-  \
CASE                              TIME                                       
dual_20s_Everywhere_14s_Jetty_10k 0.00       53.34       53.34       53.34   
                                  0.00       53.34       53.34       53.34   
                                  0.00       53.34       53.34       53.34   
                                  0.00       53.34       53.34       53.34   
                                  0.00       53.34       53.34       53.34   
...                                            ...         ...         ...   
single_22s_all_XV_12k             7.17       66.13       66.13       66.13   
                                  7.25       66.13       66.13       66.13   
                                  7.33       66.12       66.12       66.12   
                                  7.43       66.12       66.12       66.12   
                                  7.57       66.12       66.12       66.12   

                                        XV24581:P-  XV24681:P-  XV24571:P-  \
CASE                              TIME                                       
dual_20s_Everywhere_14s_Jetty_10k 0.00       55.37       51.29       55.30   
                                  0.00       55.37       51.29       55.30   
                                  0.00       55.37       51.29       55.30   
                                  0.00       55.37       51.29       55.30   
                                  0.00       55.37       51.29       55.30   
...                                            ...         ...         ...   
single_22s_all_XV_12k             7.17      171.80      167.38      113.50   
                                  7.25      171.79      167.38      113.50   
                                  7.33      171.79      167.37      113.50   
                                  7.43      171.78      167.37      113.51   
                                  7.57      171.77      167.36      113.51   

                                        XV24671:P-  ...  \
CASE                              TIME              ...   
dual_20s_Everywhere_14s_Jetty_10k 0.00       51.25  ...   
                                  0.00       51.25  ...   
                                  0.00       51.25  ...   
                                  0.00       51.25  ...   
                                  0.00       51.25  ...   
...                                            ...  ...   
single_22s_all_XV_12k             7.17      109.06  ...   
                                  7.25      109.07  ...   
                                  7.33      109.07  ...   
                                  7.43      109.07  ...   
                                  7.57      109.08  ...   

                                        LOADINGARMS:PMAX.HVAL  \
CASE                              TIME                          
dual_20s_Everywhere_14s_Jetty_10k 0.00                  49.28   
                                

Result: DataFrames showing Max and Min values for each feature per case

In [6]:
PrepMyCSV.all_min

,XV24541:P-,XV24521:P-,XV24501:P-,XV24641:P-,XV24621:P-,XV24601:P-,XV24581:P-,XV24681:P-,XV24571:P-,XV24671:P-,...,LOADINGARMS:MASP.HDIF,EXTRA:PMAX.HVAL,EXTRA:MASP.HDIF,INTERTANK:PMAX.HVAL,INTERTANK:MASP.HDIF,SHIPPIPING:PMAX.HVAL,SHIPPIPING:MASP.HDIF,MM_SYSTEM:PMAX.HVAL,MM_SYSTEM:MASP.HDIF,INCLUDE
CASE,,,,,,,,,,,,,,,,,,,,,
dual_20s_Everywhere_14s_Jetty_10k,-0.40,-1.29,-3.54,53.33,53.33,53.33,55.37,51.29,55.30,51.25,...,245.22,53.88,173.02,228.19,175.82,14.86,274.89,233.66,68.89,1
dual_20s_Everywhere_14s_Jetty_12k,-0.11,0.80,0.49,66.96,66.96,66.96,69.26,65.30,69.14,65.24,...,229.29,70.09,126.34,226.37,130.56,15.59,269.30,230.58,71.97,1
dual_20s_Everywhere_14s_Tank_10k,28.40,28.43,28.43,53.33,53.33,53.33,5.44,0.37,5.00,1.11,...,240.29,53.88,253.59,228.20,211.89,14.86,269.14,233.66,68.89,1
dual_20s_Everywhere_14s_Tank_12k,27.93,27.93,27.91,66.96,66.96,66.96,4.93,0.59,4.77,0.89,...,224.30,70.09,228.84,226.37,213.71,15.59,261.13,230.59,71.97,1
dual_20s_Everywhere_16s_Jetty_10k,-0.26,-1.93,-0.25,53.33,53.33,53.33,55.37,51.29,55.30,51.25,...,236.82,53.88,171.78,228.19,172.72,14.86,265.60,233.66,68.89,1
dual_20s_Everywhere_16s_Jetty_12k,0.13,0.80,1.24,66.96,66.96,66.96,69.26,65.30,69.14,65.24,...,219.74,70.09,137.42,226.37,141.45,15.59,255.58,230.58,71.97,1
dual_20s_Everywhere_16s_Tank_10k,36.72,36.75,36.74,53.33,53.33,53.33,5.35,0.04,4.69,1.04,...,231.63,53.88,258.49,228.20,211.89,14.86,257.82,233.66,68.89,1
dual_20s_Everywhere_16s_Tank_12k,37.42,37.43,37.40,66.96,66.96,66.96,4.99,0.64,4.25,1.19,...,213.79,70.09,234.73,226.37,213.71,15.59,246.75,230.59,71.97,1
dual_20s_Everywhere_18s_Jetty_10k,1.87,0.03,0.68,53.33,53.33,53.33,55.37,51.29,55.30,51.25,...,225.11,53.88,194.69,228.19,196.81,14.86,248.52,233.66,68.89,1


In [7]:
PrepMyCSV.all_max

,XV24541:P-,XV24521:P-,XV24501:P-,XV24641:P-,XV24621:P-,XV24601:P-,XV24581:P-,XV24681:P-,XV24571:P-,XV24671:P-,...,LOADINGARMS:MASP.HDIF,EXTRA:PMAX.HVAL,EXTRA:MASP.HDIF,INTERTANK:PMAX.HVAL,INTERTANK:MASP.HDIF,SHIPPIPING:PMAX.HVAL,SHIPPIPING:MASP.HDIF,MM_SYSTEM:PMAX.HVAL,MM_SYSTEM:MASP.HDIF,INCLUDE
CASE,,,,,,,,,,,,,,,,,,,,,
dual_20s_Everywhere_14s_Jetty_10k,59.83,59.80,59.83,53.34,53.34,53.34,278.98,274.55,282.43,277.83,...,253.28,267.05,386.19,264.26,211.89,27.69,287.72,281.62,68.89,1
dual_20s_Everywhere_14s_Jetty_12k,75.59,75.56,75.60,66.98,66.98,66.98,334.73,330.16,338.26,333.79,...,241.00,313.73,369.98,309.52,213.71,33.29,286.99,337.35,71.97,1
dual_20s_Everywhere_14s_Tank_10k,64.98,64.94,64.96,53.34,53.34,53.34,64.26,68.53,64.07,91.01,...,253.28,186.49,386.19,228.20,211.89,33.44,287.72,407.13,68.89,1
dual_20s_Everywhere_14s_Tank_12k,82.13,82.09,82.11,66.98,66.98,66.98,80.55,76.16,80.30,215.60,...,241.00,211.24,369.98,226.37,213.71,41.46,286.99,422.11,71.97,1
dual_20s_Everywhere_16s_Jetty_10k,68.91,68.90,68.95,53.34,53.34,53.34,283.17,278.74,286.17,281.62,...,253.28,268.30,386.19,267.36,211.89,36.98,287.72,285.36,68.89,1
dual_20s_Everywhere_16s_Jetty_12k,87.25,87.22,87.27,66.98,66.98,66.98,334.04,329.39,337.16,331.99,...,241.00,302.65,369.98,298.63,213.71,47.00,286.99,336.28,71.97,1
dual_20s_Everywhere_16s_Tank_10k,76.02,76.02,76.14,53.34,53.34,53.34,73.23,70.64,73.02,92.36,...,253.28,181.58,386.19,228.20,211.89,44.77,287.72,408.66,68.89,1
dual_20s_Everywhere_16s_Tank_12k,95.01,95.01,95.15,66.98,66.98,66.98,91.58,86.85,91.24,200.26,...,241.00,205.35,369.98,226.37,213.71,55.84,286.99,396.39,71.97,1
dual_20s_Everywhere_18s_Jetty_10k,85.74,85.73,85.82,53.34,53.34,53.34,280.58,275.58,282.49,276.83,...,253.28,245.39,386.19,243.27,211.89,54.07,287.72,282.38,68.89,1


### Report showing max pressure for each case

Recap: 
* For features ending with :PMAX.HVAL, report the max values and the case associated for each files.

In [8]:
PrepMyCSV.mm_report

,TANKA_RD,TANKB_RD,TANKC_RD,RUNDOWNLINES,T1_MAIN,T2_MAIN,T3_MAIN,COOLDOWNLINES,LOADINGLINES_RD,T1_COOLDOWN,...,TANKB,TANKC,LOADINGLINES,WESTJETTY,EASTJETTY,LOADINGARMS,EXTRA,INTERTANK,SHIPPIPING,MM_SYSTEM
CASE,,,,,,,,,,,,,,,,,,,,,
dual_20s_Everywhere_14s_Jetty_10k,227.36,231.39,229.41,231.57,233.57,233.66,233.61,232.21,"-9,999,999,778,196,308,361,216.00",232.20,...,59.04,242.36,281.57,53.34,58.84,57.34,267.05,264.26,27.69,281.62
dual_20s_Everywhere_14s_Jetty_12k,225.44,228.43,228.43,230.59,230.59,230.59,230.59,219.11,"-9,999,999,778,196,308,361,216.00",230.59,...,65.55,301.05,337.60,66.98,75.59,73.29,313.73,309.52,33.29,337.35
dual_20s_Everywhere_14s_Tank_10k,227.36,231.39,229.41,231.57,233.57,233.66,233.61,232.21,"-9,999,999,778,196,308,361,216.00",232.20,...,59.04,168.10,187.55,53.34,64.45,62.27,186.49,228.20,33.44,407.13
dual_20s_Everywhere_14s_Tank_12k,225.44,228.43,228.43,230.59,230.59,230.59,230.59,219.11,"-9,999,999,778,196,308,361,216.00",230.59,...,65.55,201.28,211.20,66.98,81.21,78.26,211.24,226.37,41.46,422.11
dual_20s_Everywhere_16s_Jetty_10k,227.36,231.39,229.41,231.57,233.57,233.66,233.61,232.21,"-9,999,999,778,196,308,361,216.00",232.20,...,59.04,246.85,285.26,53.34,68.75,65.76,268.30,267.36,36.98,285.36
dual_20s_Everywhere_16s_Jetty_12k,225.44,228.43,228.43,230.59,230.59,230.59,230.59,219.11,"-9,999,999,778,196,308,361,216.00",230.59,...,65.55,289.88,336.74,66.98,86.88,82.84,302.65,298.63,47.00,336.28
dual_20s_Everywhere_16s_Tank_10k,227.36,231.39,229.41,231.57,233.57,233.66,233.61,232.21,"-9,999,999,778,196,308,361,216.00",232.20,...,59.04,167.25,182.49,53.34,75.38,70.93,181.58,228.20,44.77,408.66
dual_20s_Everywhere_16s_Tank_12k,225.44,228.43,228.43,230.59,230.59,230.59,230.59,219.11,"-9,999,999,778,196,308,361,216.00",230.59,...,65.55,193.73,204.54,66.98,95.12,88.77,205.35,226.37,55.84,396.39
dual_20s_Everywhere_18s_Jetty_10k,227.36,231.39,229.41,231.57,233.57,233.66,233.61,232.21,"-9,999,999,778,196,308,361,216.00",232.20,...,59.04,220.00,281.38,53.34,85.82,77.47,245.39,243.27,54.07,282.38


### Report showing max and min force for each segment for each case

* For features ending with _F_LBF, report the max and min values and the case associated with it out of all the files.

In [9]:
PrepMyCSV.force_report

,With Flow,Against Flow,Worst Force,With Flow,Against Flow
,kips,kips,kips,Controlling Case,Controlling Case
CDML_LDJN_6,0.00,0.00,0.00,single_22s_all_XV_10k,dual_20s_Everywhere_22s_Tank_10k
CDML_LDJN_7,0.00,0.09,0.09,dual_20s_Everywhere_26s_Jetty_12k,dual_20s_Everywhere_14s_Tank_10k
CDML_LDJN_8,0.00,0.00,0.00,dual_20s_Everywhere_26s_Jetty_12k,dual_20s_Everywhere_26s_Jetty_10k
CDML_LDJN_9,1.93,0.00,1.93,dual_20s_Everywhere_14s_Jetty_10k,dual_20s_Everywhere_14s_Tank_10k
CDML_LDJN_10,0.00,0.04,0.04,dual_20s_LA_23s_LL_12k,single_20s_Everywhere_16s_Jetty_12k
...,...,...,...,...,...
JHB6_JHB8_154_155A,0.00,1.11,1.11,dual_20s_Everywhere_14s_Jetty_10k,dual_20s_Everywhere_14s_Jetty_10k
JHB6_JHB8_154_155B,2.71,0.00,2.71,dual_20s_Everywhere_22s_Jetty_12k,dual_20s_Everywhere_14s_Jetty_12k
JHC6_JHC8_163_164,4.71,0.63,4.71,dual_20s_Everywhere_14s_Jetty_10k,single_20s_LA_24s_LL_10k


Filter by With Flow, from high to low

In [10]:
PrepMyCSV.force_report.sort_values(by=[('With Flow', 'kips')], ascending=0).head()

,With Flow,Against Flow,Worst Force,With Flow,Against Flow
,kips,kips,kips,Controlling Case,Controlling Case
JHA6_JHA8_63,91.71,0.00,91.71,single_20s_Everywhere_16s_Jetty_12k,single_20s_Everywhere_16s_Jetty_12k
XVL1_JHL6_121,91.66,0.00,91.66,single_20s_LA_24s_LL_12k,dual_20s_Everywhere_14s_Jetty_12k
XVL1_JHL6_123,89.39,0.00,89.39,single_20s_LA_24s_LL_12k,dual_20s_Everywhere_14s_Jetty_12k
XVR1_JHR6_85,84.31,0.00,84.31,single_20s_Everywhere_16s_Jetty_12k,single_20s_Everywhere_16s_Jetty_12k
XVL1_JHL6_129,84.21,0.00,84.21,single_20s_LA_24s_LL_12k,dual_20s_Everywhere_14s_Jetty_12k


In [11]:
PrepMyCSV.force_report.sort_values(by=[('Against Flow', 'kips')], ascending=0).head()

,With Flow,Against Flow,Worst Force,With Flow,Against Flow
,kips,kips,kips,Controlling Case,Controlling Case
ML12_TKMF_186,0.00,45.18,45.18,single_22s_all_XV_12k,single_20s_Everywhere_14s_Tank_12k
P1A2_J1A2_6,0.00,37.85,37.85,dual_20s_LA_23s_LL_12k,dual_20s_Everywhere_16s_Jetty_10k
P1A1_J1A1_1,0.00,37.85,37.85,dual_20s_Everywhere_14s_Jetty_12k,dual_20s_Everywhere_16s_Jetty_10k
P1C3_J1C3_51,0.00,37.82,37.82,single_20s_Everywhere_14s_Tank_12k,dual_20s_Everywhere_16s_Jetty_10k
P1C1_J1C1_41,0.00,37.82,37.82,single_20s_Everywhere_14s_Tank_12k,dual_20s_Everywhere_16s_Jetty_10k


## Case Selection - Use of update()

All cases being evaluated

In [12]:
PrepMyCSV.selected_case

,CASE,INCLUDE
0,dual_20s_Everywhere_14s_Jetty_10k,1
1,dual_20s_Everywhere_14s_Jetty_12k,1
2,dual_20s_Everywhere_14s_Tank_10k,1
3,dual_20s_Everywhere_14s_Tank_12k,1
4,dual_20s_Everywhere_16s_Jetty_10k,1
5,dual_20s_Everywhere_16s_Jetty_12k,1
6,dual_20s_Everywhere_16s_Tank_10k,1
7,dual_20s_Everywhere_16s_Tank_12k,1
8,dual_20s_Everywhere_18s_Jetty_10k,1
9,dual_20s_Everywhere_18s_Jetty_12k,1


Case selection

In [13]:
PrepMyCSV.selected_case["INCLUDE"].iloc[5]=0
PrepMyCSV.selected_case

,CASE,INCLUDE
0,dual_20s_Everywhere_14s_Jetty_10k,1
1,dual_20s_Everywhere_14s_Jetty_12k,1
2,dual_20s_Everywhere_14s_Tank_10k,1
3,dual_20s_Everywhere_14s_Tank_12k,1
4,dual_20s_Everywhere_16s_Jetty_10k,1
5,dual_20s_Everywhere_16s_Jetty_12k,0
6,dual_20s_Everywhere_16s_Tank_10k,1
7,dual_20s_Everywhere_16s_Tank_12k,1
8,dual_20s_Everywhere_18s_Jetty_10k,1
9,dual_20s_Everywhere_18s_Jetty_12k,1


In [14]:
PrepMyCSV.update()

Processing Finished!
Min Flow is reported as absolute value. Actual Min Flow values are negative. If positive, it is set to be 0.
Likewise, if With Flow is negative, it is set to be 0.
Update finished!


In [15]:
PrepMyCSV.mm_report

,TANKA_RD,TANKB_RD,TANKC_RD,RUNDOWNLINES,T1_MAIN,T2_MAIN,T3_MAIN,COOLDOWNLINES,LOADINGLINES_RD,T1_COOLDOWN,...,TANKB,TANKC,LOADINGLINES,WESTJETTY,EASTJETTY,LOADINGARMS,EXTRA,INTERTANK,SHIPPIPING,MM_SYSTEM
CASE,,,,,,,,,,,,,,,,,,,,,
dual_20s_Everywhere_14s_Jetty_10k,227.36,231.39,229.41,231.57,233.57,233.66,233.61,232.21,"-9,999,999,778,196,308,361,216.00",232.20,...,59.04,242.36,281.57,53.34,58.84,57.34,267.05,264.26,27.69,281.62
dual_20s_Everywhere_14s_Jetty_12k,225.44,228.43,228.43,230.59,230.59,230.59,230.59,219.11,"-9,999,999,778,196,308,361,216.00",230.59,...,65.55,301.05,337.60,66.98,75.59,73.29,313.73,309.52,33.29,337.35
dual_20s_Everywhere_14s_Tank_10k,227.36,231.39,229.41,231.57,233.57,233.66,233.61,232.21,"-9,999,999,778,196,308,361,216.00",232.20,...,59.04,168.10,187.55,53.34,64.45,62.27,186.49,228.20,33.44,407.13
dual_20s_Everywhere_14s_Tank_12k,225.44,228.43,228.43,230.59,230.59,230.59,230.59,219.11,"-9,999,999,778,196,308,361,216.00",230.59,...,65.55,201.28,211.20,66.98,81.21,78.26,211.24,226.37,41.46,422.11
dual_20s_Everywhere_16s_Jetty_10k,227.36,231.39,229.41,231.57,233.57,233.66,233.61,232.21,"-9,999,999,778,196,308,361,216.00",232.20,...,59.04,246.85,285.26,53.34,68.75,65.76,268.30,267.36,36.98,285.36
dual_20s_Everywhere_16s_Tank_10k,227.36,231.39,229.41,231.57,233.57,233.66,233.61,232.21,"-9,999,999,778,196,308,361,216.00",232.20,...,59.04,167.25,182.49,53.34,75.38,70.93,181.58,228.20,44.77,408.66
dual_20s_Everywhere_16s_Tank_12k,225.44,228.43,228.43,230.59,230.59,230.59,230.59,219.11,"-9,999,999,778,196,308,361,216.00",230.59,...,65.55,193.73,204.54,66.98,95.12,88.77,205.35,226.37,55.84,396.39
dual_20s_Everywhere_18s_Jetty_10k,227.36,231.39,229.41,231.57,233.57,233.66,233.61,232.21,"-9,999,999,778,196,308,361,216.00",232.20,...,59.04,220.00,281.38,53.34,85.82,77.47,245.39,243.27,54.07,282.38
dual_20s_Everywhere_18s_Jetty_12k,225.44,228.43,228.43,230.59,230.59,230.59,230.59,219.11,"-9,999,999,778,196,308,361,216.00",230.59,...,65.55,247.76,316.83,66.98,107.08,97.66,267.54,266.60,67.19,317.67


Let's try sorting With Flow again

In [16]:
PrepMyCSV.force_report.sort_values(by=[('With Flow', 'kips')], ascending=0).head()

,With Flow,Against Flow,Worst Force,With Flow,Against Flow
,kips,kips,kips,Controlling Case,Controlling Case
JHA6_JHA8_63,91.71,0.00,91.71,single_20s_Everywhere_16s_Jetty_12k,single_20s_Everywhere_16s_Jetty_12k
XVL1_JHL6_121,91.66,0.00,91.66,single_20s_LA_24s_LL_12k,dual_20s_Everywhere_14s_Jetty_12k
XVL1_JHL6_123,89.39,0.00,89.39,single_20s_LA_24s_LL_12k,dual_20s_Everywhere_14s_Jetty_12k
XVR1_JHR6_85,84.31,0.00,84.31,single_20s_Everywhere_16s_Jetty_12k,single_20s_Everywhere_16s_Jetty_12k
XVL1_JHL6_129,84.21,0.00,84.21,single_20s_LA_24s_LL_12k,dual_20s_Everywhere_14s_Jetty_12k


Return all cases in the evaluation

In [17]:
PrepMyCSV.selected_case["INCLUDE"]=1
PrepMyCSV.update()

Processing Finished!
Min Flow is reported as absolute value. Actual Min Flow values are negative. If positive, it is set to be 0.
Likewise, if With Flow is negative, it is set to be 0.
Update finished!


In [18]:
PrepMyCSV.mm_report

,TANKA_RD,TANKB_RD,TANKC_RD,RUNDOWNLINES,T1_MAIN,T2_MAIN,T3_MAIN,COOLDOWNLINES,LOADINGLINES_RD,T1_COOLDOWN,...,TANKB,TANKC,LOADINGLINES,WESTJETTY,EASTJETTY,LOADINGARMS,EXTRA,INTERTANK,SHIPPIPING,MM_SYSTEM
CASE,,,,,,,,,,,,,,,,,,,,,
dual_20s_Everywhere_14s_Jetty_10k,227.36,231.39,229.41,231.57,233.57,233.66,233.61,232.21,"-9,999,999,778,196,308,361,216.00",232.20,...,59.04,242.36,281.57,53.34,58.84,57.34,267.05,264.26,27.69,281.62
dual_20s_Everywhere_14s_Jetty_12k,225.44,228.43,228.43,230.59,230.59,230.59,230.59,219.11,"-9,999,999,778,196,308,361,216.00",230.59,...,65.55,301.05,337.60,66.98,75.59,73.29,313.73,309.52,33.29,337.35
dual_20s_Everywhere_14s_Tank_10k,227.36,231.39,229.41,231.57,233.57,233.66,233.61,232.21,"-9,999,999,778,196,308,361,216.00",232.20,...,59.04,168.10,187.55,53.34,64.45,62.27,186.49,228.20,33.44,407.13
dual_20s_Everywhere_14s_Tank_12k,225.44,228.43,228.43,230.59,230.59,230.59,230.59,219.11,"-9,999,999,778,196,308,361,216.00",230.59,...,65.55,201.28,211.20,66.98,81.21,78.26,211.24,226.37,41.46,422.11
dual_20s_Everywhere_16s_Jetty_10k,227.36,231.39,229.41,231.57,233.57,233.66,233.61,232.21,"-9,999,999,778,196,308,361,216.00",232.20,...,59.04,246.85,285.26,53.34,68.75,65.76,268.30,267.36,36.98,285.36
dual_20s_Everywhere_16s_Jetty_12k,225.44,228.43,228.43,230.59,230.59,230.59,230.59,219.11,"-9,999,999,778,196,308,361,216.00",230.59,...,65.55,289.88,336.74,66.98,86.88,82.84,302.65,298.63,47.00,336.28
dual_20s_Everywhere_16s_Tank_10k,227.36,231.39,229.41,231.57,233.57,233.66,233.61,232.21,"-9,999,999,778,196,308,361,216.00",232.20,...,59.04,167.25,182.49,53.34,75.38,70.93,181.58,228.20,44.77,408.66
dual_20s_Everywhere_16s_Tank_12k,225.44,228.43,228.43,230.59,230.59,230.59,230.59,219.11,"-9,999,999,778,196,308,361,216.00",230.59,...,65.55,193.73,204.54,66.98,95.12,88.77,205.35,226.37,55.84,396.39
dual_20s_Everywhere_18s_Jetty_10k,227.36,231.39,229.41,231.57,233.57,233.66,233.61,232.21,"-9,999,999,778,196,308,361,216.00",232.20,...,59.04,220.00,281.38,53.34,85.82,77.47,245.39,243.27,54.07,282.38


## Plotting Results

Get segment length

In [19]:
segment_len_df = pd.read_csv('E:/sktime demo/Imported_Data/Segment_Length.csv', header=[0,1], index_col=0)
segment_len_df=segment_len_df.dropna()
segment_len_df.pop(("Include", "Segment"))
segment_len_df.head()

Segment,Length
Name,ft
J1A1_JHA1_2,40.53
CNA1_JHA1_3,8.42
CNA1_JHA1_4,36.30
CNA1_JHA1_5,4.17
J1A2_JHA2_7,41.02


Merge length and force into 1 df

In [20]:
segment_force_df = PrepMyCSV.force_report[[('Worst Force', 'kips')]]
MyResult = pd.concat([segment_len_df, segment_force_df], axis=1, join='inner')
MyResult.columns=MyResult.columns.droplevel(-1)
MyResult.rename(columns={"Length":"Length (ft)", "Worst Force": "Worst Force (kips)"}, inplace=True)
# Remove cases where worst forces is 0
new_col = np.empty(len(MyResult["Length (ft)"]), dtype="U100")
new_col.fill("evoleap")
MyResult["Type"] = new_col
MyResult.sort_values(by=[('Length (ft)')], ascending=0).head()

Segment,Length (ft),Worst Force (kips),Type
XVE1_JHE8_175,791.82,26.42,evoleap
XVE2_JHE2_187,772.66,40.88,evoleap
XVL1_JHL6_124,707.00,44.24,evoleap
XVR1_JHR6_80,704.00,68.53,evoleap
JHL2_JHL6_119_121,597.23,39.32,evoleap


Get Someone Result

In [21]:
other_result = pd.read_csv('E:/sktime demo/Imported_Data/Someone_Result.csv', index_col='Segment')
other_result=other_result.dropna()
new_col = np.empty(len(other_result["Length (ft)"]), dtype="U100")
new_col.fill("Bechtel")
other_result["Type"] = new_col
other_result.head()

,Length (ft),Worst Force (kips),Type
Segment,,,
J1A1 _JHA1 _2,43.46,46.10,Bechtel
CNA1 _JHA1 _3,8.41,46.10,Bechtel
CNA1 _JHA1 _4,36.31,46.60,Bechtel
CNA1 _JHA1 _5,4.16,45.70,Bechtel
J1A2 _JHA2 _7,41.35,46.10,Bechtel


Combine My Method and Other Method into 1 DataFrame for PLotly

In [22]:
# Combine Myresult and Other result and reset index
combined_df=pd.concat([other_result, MyResult], join="outer", axis=0)
combined_df

,Length (ft),Worst Force (kips),Type
J1A1 _JHA1 _2,43.46,46.10,Bechtel
CNA1 _JHA1 _3,8.41,46.10,Bechtel
CNA1 _JHA1 _4,36.31,46.60,Bechtel
CNA1 _JHA1 _5,4.16,45.70,Bechtel
J1A2 _JHA2 _7,41.35,46.10,Bechtel
...,...,...,...
RE5C_RECC_779,7.73,0.02,evoleap
RE5C_RECC_780,10.07,0.01,evoleap
RE5B_RECB_781,8.00,0.04,evoleap
RE5B_RECB_782,4.50,0.01,evoleap


## Plot in plotly

In [23]:
import plotly.express as px

df = combined_df
fig = px.scatter(df, x="Length (ft)", y="Worst Force (kips)", facet_col="Type", color="Type", trendline="ols")
fig.update_layout(title_text='Worst Force as a Function of Segment Length', title_x=0.5)
fig.show()

results = px.get_trendline_results(fig)
print(results)

print(results.px_fit_results.iloc[0].summary())
print(results.px_fit_results.iloc[1].summary())


      Type                                     px_fit_results
0  Bechtel  <statsmodels.regression.linear_model.Regressio...
1  evoleap  <statsmodels.regression.linear_model.Regressio...
                            OLS Regression Results                            
Dep. Variable:                      y   R-squared:                       0.033
Model:                            OLS   Adj. R-squared:                  0.026
Method:                 Least Squares   F-statistic:                     5.177
Date:                Mon, 09 May 2022   Prob (F-statistic):             0.0243
Time:                        22:26:22   Log-Likelihood:                -616.18
No. Observations:                 156   AIC:                             1236.
Df Residuals:                     154   BIC:                             1242.
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef   

## Alternatively, we can plot these 2 plots together

In [24]:
import plotly.express as px

df = combined_df
fig = px.scatter(df, x="Length (ft)", y="Worst Force (kips)", color="Type", trendline="ols")
fig.update_layout(title_text='Worst Force as a Function of Segment Length', title_x=0.5)
fig.show()

results = px.get_trendline_results(fig)
print(results)

print(results.px_fit_results.iloc[0].summary())
print(results.px_fit_results.iloc[1].summary())


      Type                                     px_fit_results
0  Bechtel  <statsmodels.regression.linear_model.Regressio...
1  evoleap  <statsmodels.regression.linear_model.Regressio...
                            OLS Regression Results                            
Dep. Variable:                      y   R-squared:                       0.033
Model:                            OLS   Adj. R-squared:                  0.026
Method:                 Least Squares   F-statistic:                     5.177
Date:                Mon, 09 May 2022   Prob (F-statistic):             0.0243
Time:                        22:26:22   Log-Likelihood:                -616.18
No. Observations:                 156   AIC:                             1236.
Df Residuals:                     154   BIC:                             1242.
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef   